In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast
import time
today = time.strftime("%Y_%m_%d")
from enrich_dimensions.rounds import rounds, rounding_inch_feet,rounding_lbs,rounding_w,rounding_oz, rounding_lb,rounding_gal, re_extract, curate, round_string_float, clean_data,reg_ex,rounding_period_after_unit    
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

# parameters
customer_id = '5'
formatted_attribute = 'Length'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
curation_col = f'Q:{attribute}'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name}


def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r', encoding='utf-8') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  
    
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
    
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    
#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

from fractions import Fraction
def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))
# measure='This is a fraction: 100 1/64'
# [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

import json
print('starting')
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
print(len('dfs'))
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['SKU_DESCRIPTION']

df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
print(len(df))
reg_ex(df)

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']
df.drop_duplicates(subset='external_id', inplace=True)
import collections
print('REPEATING External IDs: '+str([item for item, count in collections.Counter(df['external_id']).items() if count > 1]))

import time
today = time.strftime("%Y_%m_%d")
print(today)



starting
3
63627
REPEATING External IDs: []
2023_04_25


In [2]:
df=df[(df['mm_long'].astype(str)=='[]')&(df['mm_name'].astype(str)=='[]')]

In [3]:
print('Begin:')

Begin:


In [4]:
pat_na=r'''(?i)(\d\\?'(?!\]))|()'''
df['ft_long'] = df['long_desc'].apply(lambda x: re_extract(pat_na, x))
df['ft_name'] = df['product_name'].apply(lambda x: re_extract(pat_na, x))

# Length

In [5]:
# length
# height
# na_matches
# trips
# trip_height


In [6]:
length=df[(df['length'].astype(str)!='[]')]
length['matches_height']=length['length'].apply(lambda x: natsorted(x))
length['matches_height']=length['matches_height'].apply(lambda x: re.sub(r'in\. Length','',str(x)))
clean_data(length,'matches_height')
length['two']=length['two'].str[2:-2].str.split('","').apply(lambda x: '["'+'","'.join(natsorted(x))+'"]')
lengths=curate(length, 'two',formatted_attribute)

lengths[curation_col]=lengths[curation_col].apply(lambda x: re.sub(r'(?i)\-?foot\s?in',' ft',str(x))).apply(lambda x: re.sub('ft\s\w\w','ft',str(x))).apply(lambda x: re.sub(r'\-','',str(x))).apply(lambda x: re.sub(r'ft.*"','ft"',str(x))).apply(lambda x: re.sub(r'  ',' ',str(x))).apply(lambda x: re.sub(r'(?<!\d)0(?!\d)(?!\.) L','',str(x))).apply(lambda x: re.sub(r'engt','in',str(x))).apply(lambda x: re.sub(r'\s?in\s?in',' in',str(x))).apply(lambda x: re.sub(r'\s0\s','',str(x))).apply(lambda x: re.sub(r'(?i)\s?xLin\s?','',str(x))).apply(lambda x: re.sub(r'\s?xL\s?','',str(x))).apply(lambda x: re.sub(r'((?<=\d)in|\s?inngt in)|(\s?inL)',' in',str(x)))
print(len(lengths))
rounding_inch_feet(lengths, curation_col,'a-eghj-mo-su-z')
lengths=lengths[lengths['rounding'].astype(str)=='[]']
print(len(lengths))
match_length=lengths[['external_id',curation_col]]
match_length=match_length[(match_length['external_id'].astype(str)!='18443651')]
print(len(match_length))

Repeated External IDs: []
1480
1466
1466


# Heightz

In [7]:
heightz=df[(df['height'].astype(str)!='[]')&(df['length'].astype(str)=='[]')]
print(len(heightz))
heightz['height']=heightz['height'].apply(lambda x: re.sub(r'(\\\\?")|(\.\s(?:in|H))','in',str(x))).apply(lambda x: re.sub(r'''(\s?"\s??\,\s?"\s?)''','","',str(x))).apply(lambda x: re.sub(r'''\[(?:'|")?''','["',str(x))).apply(lambda x: re.sub(r'''(?:'|")?\]''','"]',str(x))).apply(lambda x: re.sub(r'''('?\s?"'?,(?!")(?!')\s?'?)''','","',str(x))).apply(lambda x: re.sub(r"((?<=\[)\s?')|('\s?(?=\]))|('\s?(?=\,))|((?<=\,)\s?')",'"',str(x)))
clean_data(heightz,'height')
curate(heightz, 'two',formatted_attribute)
heightz[curation_col]=heightz[curation_col].str[2:-2].str.split('","').apply(lambda x: '["'+'","'.join(natsorted(x))+'"]')
heightz[curation_col]=heightz[curation_col].apply(lambda x: re.sub(r'2625','26.25',str(x))).apply(lambda x: re.sub(r'(?<=\d)"\]',' in"]',str(x))).apply(lambda x: re.sub(r'(?<=\d)"',' in"',str(x))).apply(lambda x: re.sub(r'ft\s?in','ft',str(x))).apply(lambda x: re.sub(r'"\]"\]','"]"',str(x))).apply(lambda x: re.sub(r'″',' in"',str(x))).apply(lambda x: re.sub(r'("?\,"?)','","',str(x))).apply(lambda x: re.sub(r'(?<=\d)Foot',' ft',str(x)))                                   
print(len(heightz))
rounding_inch_feet(heightz, curation_col,'a-eghj-mo-su-z')
heightz=heightz[heightz['rounding'].astype(str)=='[]']
print(len(heightz))
matchesheight=heightz[['external_id',curation_col]]
print(len(matchesheight))

1257
Repeated External IDs: []
1257
1227
1227


# NA Matches

In [8]:
pat_na=r'''(?i)((?<!Cord Measures.{10})\d+\.\d+.{0,2}["'”].{0,2}[LWHD]?\b)|((?<!Cord Measures.{10})\d+.?\d+\/\d+.{0,2}["'”].{0,2}[LWHD]?\b)|((?<!Cord Measures.{10})(?<!\.)(?<!\/)\d+(?!\.\d).{0,2}["'”].{0,2}[LWHD]?\b)|((?<!Cord Measures.{10})(?<!\.)(?<!\/)\d+(?!\.).{0,2}["'”].{0,2}[LWHD]?\b)|((?<!\.)\d+(?!\.\d).{0,3}?diam)|((?:(?:le?n?g?t?h?)|(?:He?i?g?h?t?)|(?:Wi?d?t?h?)|(?:De?p?t?h?))\:\s\d+\s?)|()'''
df['na_matches'] = df['long_desc'].apply(lambda x: re_extract(pat_na, x))
df_na=df[(df['na_matches'].astype(str)=='[]')&(df['length'].astype(str)=='[]')&(df['height'].astype(str)=='[]')&(df['double_name'].astype(str)=='[]')&(df['double_long'].astype(str)=='[]')&(df['blank_matches_long'].astype(str)=='[]')]
df_na[curation_col]='n/a'
matchesna=df_na[['external_id',curation_col]]
print(len(matchesna))

6261


# Max Min (Length is the max)

In [9]:
pat=r'''(?i)(\d+(?:\.\d+)?\\{0,3}\s?(?:"|''|“|in(?:che?s?)|')?\s?x\s?\d+(?:\.\d+)?\\{0,3}\s?(?:"|''|“|in(?:che?s?)|')?\s?x\s?\d+(?:\.\d+)?\\{0,3}\s?(?:"|''|“|in(?:che?s?)|')?\s?(?=\(L\s?x\s?W\s?x\s?H\)))|()'''
df['trips'] = df['long_desc'].apply(lambda x: re_extract(pat, x))

pat=r'(?i)(L\s?x\s?[A-z]\s?x\s?[A-z])|()'
df['LWH'] = df['long_desc'].apply(lambda x: re_extract(pat, x))


df_wdh=df[(df['LWH'].astype(str)!='[]')&(df['na_matches'].astype(str)!='[]')&(df['height'].astype(str)=='[]')&(df['trips'].astype(str)!='[]')&(df['length'].astype(str)=='[]')]
df_name=df_wdh[['external_id','product_name','long_desc','height','length','blank_matches','blank_matches_long','tripple_name','tripple_long','double_name','na_matches','LWH','trips']]
df_name['tripps_length']=df_name['trips'].apply(lambda x: re.sub(r'''(?i)(x.*(?='\]))|(x.*(?=\]))|((?<!\d)\\(?!\d)(?!\-\d)(?! \d))|(\\)''','',str(x))).apply(lambda x: re.sub(r'''(\\{1,3}(?:"|''|“))|((?<=\d)\s?(?:"|''|“)\s?)''',' in',str(x))).apply(lambda x: re.sub(r'''((?<=)\s?'\s?)|(?<!')(?<!")(?=\,)|(?<=\[)(?!')(?!")|(?<!')(?<!")(?=\])|(?:''|“)|("")''','"',str(x)))                        
df_name[curation_col]=df_name['tripps_length']
print(len(df_name))
curate(df_name,curation_col,'length')
print(len(df_name))
rounding_inch_feet(df_name,curation_col,'a-eghj-mo-su-z')
df_name=df_name[df_name['rounding'].astype(str)=='[]']
print(len(df_name))
match_trips=df_name[['external_id',curation_col]]


1390
Repeated External IDs: []
1390
1390


In [10]:
trip_height=r'''(?i)(\d+(?:\.\d+)?\\{0,3}\s?(?:"|''|“|in(?:che?s?)|')?\s?x\s?\d+(?:\.\d+)?\\{0,3}\s?(?:"|''|“|in(?:che?s?)|')?\s?x\s?\d+(?:\.\d+)?\\{0,3}\s?(?:"|''|“|in(?:che?s?)|')?)|()'''
df['trip_height'] = df['long_desc'].apply(lambda x: re_extract(trip_height, x))
tripple=df[(df['length'].astype(str)=='[]')&(df['height'].astype(str)=='[]')&(df['LWH'].astype(str)=='[]')&(df['trips'].astype(str)=='[]')&(df['trip_height'].astype(str)!='[]')]

print(len(tripple))
tripple['trip_height_one']=tripple['trip_height'].apply(lambda x: re.sub(r'\\{0,3}"\s?',' in',str(x))).apply(lambda x: re.sub(r'(?i)(\s?in(?:che?s?)?)',' in',str(x))).apply(lambda x: re.sub(r'''((?<=)\s?'\s?)|(?<!')(?<!")(?=\,)|(?<=\[)(?!')(?!")|(?<!')(?<!")(?=\])|(?:''|“)|("")''','"',str(x))).apply(lambda x: re.sub(r'(?i)(x.*)|(\\{0,3})','',str(x)))
tripple['trip_heights']=tripple['trip_height_one'].apply(lambda x: re.sub(r"(?<=\d)\\{0,3}\s?'(?!\])",' ft',str(x)))
tripple['tripps']=tripple['trip_heights'].apply(lambda x: re.sub(r"(?<=\d)\\{0,3}'(?!\])",' ft',str(x))).apply(lambda x: re.sub(r'''(?i)(\s?x.*(?='\]))''','',str(x))).apply(lambda x: re.sub(r'','',str(x))).apply(lambda x: re.sub(r'''((?<=)\s?'\s?)|(?<!')(?<!")(?=\,)|(?<=\[)(?!')(?!")|(?<!')(?<!")(?=\])|(?:''|“)|("")''','"',str(x)))                         
curate(tripple,'tripps','length')
print(len(tripple))
rounding_inch_feet(tripple,curation_col,'a-eghj-mo-su-z')
tripple=tripple[tripple['rounding'].astype(str)=='[]']
print(len(tripple))
match_tripple=tripple[['external_id',curation_col]]
rounding_inch_feet(tripple,curation_col,'a-eghj-mo-su-z')

1516
Repeated External IDs: []
1516
1511


Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, TYPE, COLOR, COLOR_s, SKU_SIZE, SKU_TYPE, RecordType, SKU_SIZE_s, COLOR_GROUP, DESCRIPTION, COLORGROUP_s, VDC_SKU_TYPE, f_Product_Type, SKU_DESCRIPTION, LONG_DESCRIPTION, ROLLUP_TYPE_CODE, s_f_Product_Type, PERSONALIZATION_TYPE, gbi_exp_product_type, gbi_syn_product_type, MARKETPLACE_ITEM_FLAG, ASSEMBLEDPRODUCTWIDTHIN, ASSEMBLEDPRODUCTHEIGHTIN, ASSEMBLEDPRODUCTLENGTHIN, gbi_product_type_affinity, ASSEMBLEDPRODUCTDIAMETERIN, f_binProduct_Type, MAX_ASSEMBLEDPRODUCTWIDTHIN, MIN_ASSEMBLEDPRODUCTWIDTHIN, MAX_ASSEMBLEDPRODUCTHEIGHTIN, MAX_ASSEMBLEDPRODUCTLENGTHIN, MIN_ASSEMBLEDPRODUCTHEIGHTIN, MIN_ASSEMBLEDPRODUCTLENGTHIN, SERVICE_TYPE_CD, MAX_ASSEMBLEDPRODUCTDIAMETERIN, MIN_ASSEMBLEDPRODUCTDIAMETERIN, s_f_Made_In, s_f_Construction, s_f_Theme, s_f_College, gbi_syn_size, s_f_Team_Sports, Fabric_FiberType, s_f_Closure_Type, s_f_Fabric_FiberType, s_f_Indoor_Outdoor_Use, s_f_Maximum_Weight_Capacity_lb, s_f_Soft_Textiles_Use_and_Care_Instructions, Set_Size, s_f_Number_of_Batteries, s_f_Gender, s_f_L4_BBBY_Baby_Category, s_f_Holiday_Occasion_multi, s_f_Lifestyle_multi, Cord_Length, s_f_Power_Source_multi, s_f_Holiday_Tree_Height, s_f_Assembly_Instructions, s_f_Holiday_Timer_Included, s_f_Hardlines_Use_and_Care_multi, s_f_Tabletop_Style, s_f_Type_of_Chiller, s_f_Wattage, s_f_Control_Type, s_f_Display_Type_multi, Soft_Textiles_Material, Reversible_Fabric_Fiber, s_f_Baby_Kids_Theme_multi, s_f_Soft_Textiles_Material, Soft_Textiles_Reverse_Material, s_f_Bedding_Style, s_f_Life_Stage_multi, s_f_Shape, s_f_Fixture_Mounts, s_f_Number_of_Hooks, s_f_Wood_Color_multi, s_f_Frame_Construction, s_f_Subject_Matter_multi, s_f_Wall_Decor_Orientation, Frame_Color, s_f_Frame_Color, s_f_Frame_Finish, s_f_External_Finish, s_f_Towel_Style, s_f_Pattern_Design, s_f_Embellishment_multi, s_f_Product_Protection_Features_multi, s_f_Window_Style, s_f_Light_Filtration, s_f_Panel_Length_Rod_to_Bottom_in, s_f_Window_Hanging_Length_Rod_to_bottom_in, s_f_Exterior_Finish, s_f_Accessory_Type, Service_Size, s_f_Capacity, s_f_Service_Size, s_f_Dinnerware_Style, ...]
Index: []

In [11]:
# print(len(dubs))
# dubs[['external_id','dub','length','height','width','diameter','blank_matches','blank_matches_long','units']]

In [12]:
dubs=df[(df['length'].astype(str)=='[]')&(df['height'].astype(str)=='[]')&(df['double_name'].astype(str)=='[]')&(df['double_long'].astype(str)!='[]')&(df['ft_long'].astype(str)=='[]')&(df['ft_name'].astype(str)=='[]')]
print(len(dubs))

dubs['dub']=dubs['double_long'].apply(lambda x: re.sub(r'(\-ft)',' ft',str(x))).apply(lambda x: re.sub(r'''(?i)(\-in)|(\s?\\{0,3}\s?"\s?|(?<!\d)\s(?!\d))|([a-wyz])|(”)|(\[)|(\])|(\\{0,2}xa?0(?!\.))|(\?)|(®)|(\))|''','',str(x))).apply(lambda x: f'"[{x}]"').astype(str)
df1 = dubs['dub'].str[2:-2].str.split(',').explode().str.split(r'(?i)x', expand=True).fillna('')
df1[1]=df1[1].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))]).apply(lambda x: re.sub(r'''('?\[(?:"|')\[(?:"|'))''','["',str(x))).apply(lambda x: re.sub(r'''('\](?:"|')\])|(?:\[|\]|"|'|\\+)''','',str(x)))
df1[0]=df1[0].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))]).apply(lambda x: re.sub(r'''('?\[(?:"|')\[(?:"|'))''','["',str(x))).apply(lambda x: re.sub(r'''('\](?:"|')\])|(?:\[|\]|"|'|\\+)''','',str(x)))


x=df1[1].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)]).to_list()
y=df1[0].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)]).to_list()

dubs[['Max', 'Min']] = pd.concat([df1.astype(float).max(1).groupby(level=0).apply(list), 
                                df1.astype(float).min(1).groupby(level=0).apply(list)], axis=1)

clean_data(dubs,'Max')
dubs['doubs']=dubs['one'].apply(lambda x: re.sub(r'\[\s?\"\s?\[\s?(:"|")?','',str(x))).apply(lambda x: re.sub(r'"?\]"\]','"]',str(x))).apply(lambda x: re.sub(r'(?<=\d)"',' in"',str(x)))
curate(dubs,'doubs','length')
rounding_inch_feet(dubs,curation_col,'a-eghj-mo-su-z')
dubz=dubs[dubs['rounding'].astype(str)=='[]']
rounding_inch_feet(dubz,curation_col,'a-eghj-mo-su-z')

15065


ValueError: could not convert string to float: '4-'

In [ ]:
# df1[0][-1500:-1000]
# dubs['dub'][0:500]

In [ ]:
match_dub=dubz[['external_id',curation_col]]

In [ ]:
dub_name=df[(df['length'].astype(str)=='[]')&(df['height'].astype(str)=='[]')&(df['double_name'].astype(str)!='[]')&(df['ft_long'].astype(str)=='[]')&(df['ft_name'].astype(str)=='[]')]
print(len(dub_name))

dub_name['double_name']=dub_name['double_name'].apply(lambda x: re.sub(r'ʺ',' in',str(x))).apply(lambda x: re.sub(r'‘','',str(x)))

dub_name['dub']=dub_name['double_name'].apply(lambda x: re.sub(r'''(?i)((\-inc?h?)|((?<=\d)\-)|(inch)|(\s?\\{0,3}\s?"\s?)|((?<!\d)\s(?!\d))|([a-wyz])|(”)|(\[)|(\])|(\\{0,2}xa?0(?!\.)))|(\))|(\()''','',str(x))).apply(lambda x: f'["{x}"]').astype(str)
dub_name['dub']=dub_name['dub'].apply(lambda x: re.sub(r'''("')|('")''','"',str(x)))

df1 = dub_name['dub'].str[2:-2].str.split(',').explode().str.split(r'(?i)x', expand=True).fillna('')
df1[1]=df1[1].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))]).apply(lambda x: re.sub(r'''('?\[(?:"|')\[(?:"|'))''','["',str(x))).apply(lambda x: re.sub(r'''('\](?:"|')\])|(?:\[|\]|"|'|\\+)''','',str(x)))
df1[0]=df1[0].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))]).apply(lambda x: re.sub(r'''('?\[(?:"|')\[(?:"|'))''','["',str(x))).apply(lambda x: re.sub(r'''('\](?:"|')\])|(?:\[|\]|"|'|\\+)''','',str(x)))

x=df1[1].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)]).to_list()
y=df1[0].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)]).to_list()

dub_name[['Max', 'Min']] = pd.concat([df1.astype(float).max(1).groupby(level=0).apply(list), 
                                df1.astype(float).min(1).groupby(level=0).apply(list)], axis=1)

clean_data(dub_name,'Max')
dub_name['doubs']=dub_name['one'].apply(lambda x: re.sub(r'\[\s?\"\s?\[\s?(:"|")?','',str(x))).apply(lambda x: re.sub(r'"?\]"\]','"]',str(x)))
curate(dub_name,'doubs','length')
rounding_inch_feet(dub_name,curation_col,'a-eghj-mo-su-z')
# print(len(dub_name))
dub_name=dub_name[dub_name['rounding'].astype(str)=='[]']
print(len(dub_name))

In [ ]:
# df_na=df[(df['na_matches'].astype(str)=='[]')&(df['length'].astype(str)=='[]')&(df['height'].astype(str)=='[]')&(df['double_name'].astype(str)=='[]')&(df['double_long'].astype(str)=='[]')&(df['blank_matches_long'].astype(str)=='[]')]
# df[(df['na_matches'].astype(str)=='[]')]
# x=df[['external_id','product_name','long_desc','length','height','width','blank_matches','blank_matches_long','tripple_name','tripple_long','double_name','double_long','na_matches','LWH','trip_height']]
# x[0:500]

In [ ]:
# pat=r'(?i)((?<![A-z])[A-z]\s?x\s?[A-z]\s?x\s?H)|()'
# # pat=r'(?i)(W x D x H)|(L x W x H)|(L\,\s?W\,\s?H)|()'
# df['junk'] = df['long_desc'].apply(lambda x: re_extract(pat, x))
# junk=df[(df['junk'].astype(str)!='[]')&(df['LWH'].astype(str)=='[]')]
# print(len(df))
# print(len(junk))
# junk[0:500]

In [ ]:
pat='''(?i)(L\s?x\s?(?:H|D|W|[A-z])\s?x\s?(?:H|D|W|[A-z]))|()'''
df['LWH'] = df['long_desc'].apply(lambda x: re_extract(pat, x))
pat='''(?i)(?i)(.{0,30}L(?:ength)?\s?x\s?[A-z]{0,10}\s?x\s?H)|(.{0,30}[A-z]\s?(?<!x)x\s?[A-z]\s?(?<!x)x\s?[A-z]{0,10})|()''' 
df['match'] = df['long_desc'].apply(lambda x: re_extract(pat, x))

pat='''(?i)((?:\d+(?:\.\d+|\/\d+)?\\{0,3}\s?(?:''|"|inc?h?e?s?))\s?x\s?(?:\d+(?:\.\d+|\/\d+)?\\{0,3}\s?(?:''|"|inc?h?e?s?))\s?x\s?(?:\d+(?:\.\d+|\/\d+)?\\{0,3}\s?(?:''|"|inc?h?e?s?))[^\w]{0,10}(?:W\s?x\s?D\s?x\s?H|L\s?x\s?W\s?x\s?H|[A-z]\s?x\s?[A-z]\s?x\s?[A-z]))|()'''    
pat='''(?i)(\d+.{0,10}x.{0,3}\d.{0,10}x.{0,3}\d.{0,15}L\s?x\s?[A-z]\s?x\s?[A-z])|()'''
df['lwh'] = df['match'].apply(lambda x: re_extract(pat, str(x)))

In [ ]:
lwh=df[(df['LWH'].astype(str)!='[]')&(df['lwh'].astype(str)!='[]')&(df['trips'].astype(str)!='[]')]
print(len(df))
print(len(lwh))

In [ ]:
length_width_height=lwh[['external_id','product_name','long_desc','lwh','LWH','match']]
length_width_height[curation_col]=length_width_height['lwh'].apply(lambda x: re.sub(r'''(\\?'\\?')|((?<!\,\s)(?<!\,)(?<!\[)\\?"(?!\])(?!\,)(?!\s?\,))''',' in',str(x))).apply(lambda x: re.sub(r'(\s?\\+\s?)','',str(x))).apply(lambda x: re.sub(r'''(?i)(\s?x.{0,3}\d+.{0,10}x.{0,3}\d+.{0,17}L\s?x\s?[A-z]\s?x\s?[A-z])''',' in',str(x))).apply(lambda x: re.sub(r'''(?i)(\s?"\s?)''','"',str(x))).apply(lambda x: re.sub(r'''(?i)(\s?in\s?in)''',' in',str(x))).apply(lambda x: re.sub(r'''(?i)(\s?in\s?in)''',' in',str(x))).apply(lambda x: re.sub(r'''((?<=\[)')|('(?=\]))|((?<=\,)\s?')|('(?=\,))''','"',str(x))).apply(lambda x: re.sub(r'''((?<=\d)\.0(?!\d))''','',str(x))).apply(lambda x: re.sub(r"(?i)((?<=\d)'\s?in)|((?<=\d)\s?ft\s?in)",' ft',str(x)))                                 
length_width_height[curation_col]=length_width_height[curation_col].apply(lambda x: re.sub(r'(\s?(?<!")(?=\]))|("")','"',str(x))).apply(lambda x: re.sub(r'("")','"',str(x))).apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(lambda x: natsorted(x)).apply(lambda x: re.sub(r'''(((?:"|')?\s?\,\s?(?!")(?:')?))''','","',str(x))).apply(lambda x: re.sub(r'''((?<=\[)\s?'\s?)|(\s?'\s?(?=\]))|(\s?'\s?(?=\,))|((?<=\,)\s?'\s?)''','"',str(x)))

print(len(length_width_height))
rounding_inch_feet(length_width_height, curation_col,'a-eghj-mo-su-z')
lwhs=length_width_height[length_width_height['rounding'].astype(str)=='[]']
print(len(lwhs))

match_lwh=lwhs[['external_id',curation_col]]
print(len(match_lwh))

In [ ]:
pat='''(?i)(L\s?x\s?(?:H|D|W|[A-z])(?!\sx)(?!x)(?!\-x))|()'''
df['LW'] = df['long_desc'].apply(lambda x: re_extract(pat, x))
pat='''(?i)(.{0,30}L(?:ength)?\s?x\s?[A-z]{0,10}\s?)|(.{0,30}[A-z]\s?(?<!x)x\s?[A-z]\s?(?<!x)).{0,10}|()''' 
df['match_lw'] = df['long_desc'].apply(lambda x: re_extract(pat, x))

# pat='''(?i)((?:\d+(?:\.\d+|\/\d+)?\\{0,3}\s?(?:''|"|inc?h?e?s?))\s?x\s?(?:\d+(?:\.\d+|\/\d+)?\\{0,3}\s?(?:''|"|inc?h?e?s?))\s?x\s?(?:\d+(?:\.\d+|\/\d+)?\\{0,3}\s?(?:''|"|inc?h?e?s?))[^\w]{0,10}(?:W\s?x\s?D\s?x\s?H|L\s?x\s?W\s?x\s?H|[A-z]\s?x\s?[A-z]\s?x\s?[A-z]))|()'''    
pat='''(?i)(\d+.{0,10}x.{0,3}\d+.{0,10}L\s?x\s?[A-z]\s?)|()'''
df['lw'] = df['match'].apply(lambda x: re_extract(pat, str(x)))

In [ ]:
lwh=df[(df['LW'].astype(str)!='[]')&(df['lw'].astype(str)!='[]')&(df['trips'].astype(str)!='[]')]
print(len(df))
print(len(lwh))

In [ ]:
pat='''(?i)((?<!x)(?<!x\s)\d+(?:\.\d+|\/\d+).{0,2}L(?:ength)?\b)|()'''
df['len'] = df['long_desc'].apply(lambda x: re_extract(pat, x))
length_length=df[(df['len'].astype(str)!='[]')&(df['trips'].astype(str)=='[]')&(df['lwh'].astype(str)=='[]')&(df['length'].astype(str)=='[]')]
print(len(length_length))

In [ ]:
length_length[['external_id','product_name','long_desc','len']]

In [ ]:
pat_na=r'''(?i)(dimensions)|()'''
df['dimension'] = df['long_desc'].apply(lambda x: re_extract(pat_na, x))
dimension=df[(df['dimension'].astype(str)!='[]')&(df['LWH'].astype(str)=='[]')&(df['trips'].astype(str)=='[]')&(df['length'].astype(str)=='[]')]
print(len(df))
print(len(dimension))

In [ ]:
dimension[['external_id','product_name','long_desc','dimension']][0:500]
# dimension[0:500]

In [ ]:
# dub_name[['external_id','product_name','long_desc','Q:length']]
# dub_name.columns.to_list()

In [ ]:
match_dub_name=dub_name[['external_id',curation_col]]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesna))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesheight))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(match_length))

In [ ]:
print(len(df))
print(len(matchesna)+len(match_length))
print(len(matchesna)+len(match_length)+len(match_dub_name)+len(match_dub)+len(match_lwh))
# print(len(match_dub_name))

In [ ]:
rounding_inch_feet(lengths, curation_col,'a-eghj-mo-su-z')

In [ ]:
# # Length is greater
# x=df[(df['length'].astype(str)=='[]')&(df['diameter'].astype(str)=='[]')&(df['height'].astype(str)=='[]')&(df['na_matches'].astype(str)=='[]')]
# y=x[x['blank_matches'].astype(str)!='[]']
# print(len(y))
# y

In [ ]:
stop   

# send to the folder for upload

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-match_lwh.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute,match_lwh,today)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute,match_length,today)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute,matchesheight,today)    

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 

looks_good('Bed Bath & Beyond', attribute,matchesna,today)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False)

looks_good('Bed Bath & Beyond', attribute,match_dub,today)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False)

looks_good('Bed Bath & Beyond', attribute,match_dub_name,today)